In [73]:
import numpy as np
import pandas as pd
from scipy.stats import entropy

In [75]:
fnenm = pd.read_csv("pdm_fnenm.csv")
scaled_f_telem = pd.read_csv("abszscale_telemetry.csv")

In [76]:
OverallTelemHists = {
    "volt": [np.histogram(scaled_f_telem[["volt"]],bins=np.arange(0,6,0.01),density=True)],
    "rotate": [np.histogram(scaled_f_telem[["rotate"]],bins=np.arange(0,6,0.01),density=True)],
    "pressure": [np.histogram(scaled_f_telem[["pressure"]],bins=np.arange(0,8,0.01),density=True)],
    "vibration": [np.histogram(scaled_f_telem[["vibration"]],bins=np.arange(0,7,0.01),density=True)],
}

In [77]:
fnenm

,Unnamed: 0,datetime,machineID,errorID,failure,maint_comp
0,0,2014-06-01 06:00:00,1,NaN,NaN,comp2
1,1,2014-07-16 06:00:00,1,NaN,NaN,comp4
2,2,2014-07-31 06:00:00,1,NaN,NaN,comp3
3,3,2014-12-13 06:00:00,1,NaN,NaN,comp1
4,0,2015-01-03 07:00:00,1,error1,NaN,NaN
...,...,...,...,...,...,...
7961,3917,2015-12-08 06:00:00,100,error3,NaN,NaN
7962,760,2015-12-09 06:00:00,100,NaN,comp2,NaN
7963,3284,2015-12-09 06:00:00,100,NaN,NaN,comp2
7964,3918,2015-12-22 03:00:00,100,error3,NaN,NaN


In [78]:
scaled_f_telem

,Unnamed: 0,datetime,machineID,volt,rotate,pressure,vibration
0,0,2015-01-01 06:00:00,1,0.350769,0.533491,1.105949,0.875673
1,1,2015-01-01 07:00:00,1,0.509282,0.832626,0.488578,0.564016
2,2,2015-01-01 08:00:00,1,0.013680,1.532918,2.318900,1.155632
3,3,2015-01-01 09:00:00,1,0.536130,1.907128,0.759358,0.137260
4,4,2015-01-01 10:00:00,1,0.849031,0.213165,0.998127,2.680361
...,...,...,...,...,...,...,...
876095,876095,2016-01-01 02:00:00,100,0.558409,0.975480,0.129613,1.934123
876096,876096,2016-01-01 03:00:00,100,1.214759,0.007540,0.242388,0.979811
876097,876097,2016-01-01 04:00:00,100,1.399544,0.022998,0.608746,1.476541
876098,876098,2016-01-01 05:00:00,100,0.341891,0.623335,0.291655,0.828670


In [79]:
def getTelemetryFeatures(datetime, machineID):
    datetime = pd.to_datetime(datetime)
    telem_features = {}
    mactelem = scaled_f_telem[scaled_f_telem['machineID']==machineID]
    mactelem = mactelem[datetime - pd.to_datetime(mactelem['datetime']) <= pd.Timedelta("7 days")]
    mactelem = mactelem[datetime - pd.to_datetime(mactelem['datetime']) >= pd.Timedelta("0")]
    binNumber = {'volt':6, 'rotate':6, 'pressure':8, 'vibration':7}
    currentTelemHists = {}
    currentTelemBins = {}
    
    for metric in ['volt','rotate','pressure','vibration']:
        currentTelemHists[metric] = [np.histogram(mactelem[[metric]],bins=np.arange(0,binNumber[metric],0.01),density=True)]  
        telem_features[metric+"_KLD"] =  entropy(currentTelemHists[metric][0][0], OverallTelemHists[metric][0][0])
        telem_features[metric+"_max"] = np.max(mactelem[metric])
        currentTelemBins[metric] = [np.histogram(mactelem[[metric]],bins=np.arange(0,binNumber[metric]+1))]
        for bnm_ in range(1,binNumber[metric]):
            telem_features[metric+"_bin"+ str(bnm_) +"ct"] = currentTelemBins[metric][0][0][bnm_]
        telem_features[metric+"_avg"] = np.mean(mactelem[metric])
        telem_features[metric+"_topavg"] = np.mean(mactelem[metric].sort_values(ascending=False)[:10])  
    return telem_features

In [80]:
def sampleByMacID(machineID):
    fnenm_cur_mac = fnenm[fnenm['machineID']==machineID]
    sampleDF_curmac = pd.DataFrame()
    for comp_ in ["comp1","comp2","comp3","comp4"]:
        fnenm_cur_mac_comp = fnenm_cur_mac[fnenm_cur_mac['maint_comp']==comp_]
        last_maint = None
        for index,line in fnenm_cur_mac_comp.iterrows():
            if last_maint == None:
                fnenm_cur_mac_comp.loc[index, 'time_since_last_maint'] = None
                fnenm_cur_mac_comp.loc[index, 'error_in_last_month'] = None
            else:
                fnenm_cur_mac_comp.loc[index, 'time_since_last_maint'] = (pd.to_datetime(line['datetime']) - pd.to_datetime(last_maint)).days
                errorVec = [0,0,0,0,0]
                failureVec = [0,0,0,0]
                for index2, line2 in fnenm_cur_mac.iterrows():
                    if not pd.isna(line2['errorID']):
                        timedelta = pd.to_datetime(line['datetime']) - pd.to_datetime(line2['datetime'])
                        if timedelta >= pd.Timedelta("0") and timedelta <= pd.Timedelta("15 days"):
                            errorVec[int(line2['errorID'][-1])-1] += 1
                    if not pd.isna(line2['failure']):
                        timedelta = pd.to_datetime(line['datetime']) - pd.to_datetime(line2['datetime'])
                        if timedelta >= pd.Timedelta("0") and timedelta <= pd.Timedelta("2 days"):
                            failureVec[int(line2['failure'][-1])-1] += 1
                for _ in range(5):
                    fnenm_cur_mac_comp.loc[index, 'error'+ str(_+1) +'_in_last_month'] = int(errorVec[_])
                for _ in range(4):
                    fnenm_cur_mac_comp.loc[index, 'failure'+ str(_+1) +'_in_2_days'] = int(failureVec[_])
            telem_feats = getTelemetryFeatures(line['datetime'], machineID)
            for tf_ in telem_feats:
                fnenm_cur_mac_comp.loc[index, tf_] = telem_feats[tf_]
            fnenm_cur_mac_comp.loc[index, 'maint_comp'] = int(comp_[-1])
            last_maint = line['datetime']
        fnenm_cur_mac_comp = fnenm_cur_mac_comp[1:]
        valid_keys = ["machineID","maint_comp","error1_in_last_month","error2_in_last_month",\
                                  "error3_in_last_month","error4_in_last_month","error5_in_last_month",\
                                 "failure1_in_2_days","failure2_in_2_days","failure3_in_2_days",\
                                 "failure4_in_2_days","time_since_last_maint"]
        for tf_ in telem_feats:
            valid_keys.append(tf_)
        sampleDF_curmac = pd.concat([sampleDF_curmac,fnenm_cur_mac_comp[valid_keys]], axis=0)
    return sampleDF_curmac
        

In [82]:
def getAllMaints():
    all_df = pd.DataFrame()
    for i in range(1,101):
        df = sampleByMacID(i)
        all_df = pd.concat([all_df,df], axis=0)
    return all_df

In [83]:
res = getAllMaints()

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:905: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/Users/qhu/opt/anacon

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

...

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:905: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/Users/qhu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [84]:
res

,machineID,maint_comp,error1_in_last_month,error2_in_last_month,error3_in_last_month,error4_in_last_month,error5_in_last_month,failure1_in_2_days,failure2_in_2_days,failure3_in_2_days,...,vibration_KLD,vibration_max,vibration_bin1ct,vibration_bin2ct,vibration_bin3ct,vibration_bin4ct,vibration_bin5ct,vibration_bin6ct,vibration_avg,vibration_topavg
9,1,1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.988099,4.912058,29.0,16.0,10.0,2.0,0.0,0.0,1.506445,3.564090
12,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.667304,2.569709,34.0,10.0,0.0,0.0,0.0,0.0,0.759295,2.311729
22,1,1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.708390,2.864370,42.0,4.0,0.0,0.0,0.0,0.0,0.737314,2.052989
24,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.699894,2.752081,44.0,8.0,0.0,0.0,0.0,0.0,0.787911,2.219301
45,1,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.942832,3.866600,44.0,18.0,3.0,0.0,0.0,0.0,0.931065,2.953421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7923,100,4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.971890,3.346064,58.0,18.0,2.0,0.0,0.0,0.0,1.037328,2.733414
7929,100,4,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.672497,2.763350,46.0,6.0,0.0,0.0,0.0,0.0,0.749159,2.103722
7938,100,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.679350,2.945968,46.0,5.0,0.0,0.0,0.0,0.0,0.721410,2.084603
7952,100,4,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.702853,2.276435,35.0,4.0,0.0,0.0,0.0,0.0,0.663088,1.911586


In [85]:
res.to_csv("maints.csv")